# 基盤人工知能演習 第5回

※本演習資料の二次配布・再配布はお断り致します。

　今回から3回かけて、ニューラルネットワーク (Neural network) を用いた分類問題を学んでいく。本日の内容は以下の通りである。

- **`torch.tensor`を利用した単層パーセプトロン (Single layer perceptron) の実装**

- **`torch.nn.Linear()`を利用した単層パーセプトロンの実装**

- **GPU (Graphics Processing Unit) を用いた計算**

## AI5.1 | 最急降下法のおさらい

　前回（基盤人工知能演習 第4回）、ロジスティック回帰の内部実装で「最急降下法 (steepest gradient descent)」に基づいて重みを最適化したことを覚えているだろうか。

　**最急降下法はニューラルネットワークでも極めて重要**であるため、まずは最急降下法のおさらいをしてみる。

-----

##### 課題 AI5.1

　最急降下法は以下の式で重み $w$ を更新していくものだった。

$\begin{eqnarray}
w^{(t+1)} & = & w^{(t)} - \eta \frac{\partial L(w)}{\partial w}
\end{eqnarray}$

（ニューラルネットワークの文脈に合わせるために、 $\alpha$ を $\eta$ に書き直している）

　$w$ は1次元で $L(w) = w^2 + 2w$ であるとき、以下のコードの`__xxxxx__`, `__yyyyy__` を埋めて最急降下法を完成させよ。（ $w=-1$ に収束することを確認することで実装が正しいか推定すると良い）

In [0]:
# 勾配を計算する関数
# 自分で dL(w)/dw の微分を行い、その結果に従って gradient() を実装せよ
def gradient(w):
  return __xxxxx__;

def optimize(w_initial, eta=0.1, steps=100):
  w_now = w_initial
  for i in range(steps):
    w_now -= __yyyyy__;
  return w_now

print(optimize(1)) 

--------------

## AI5.2 | データセットの作成（前回演習と同じデータセット）

　本日は、ロジスティック回帰を学習した時と同じデータセットを作成して、これを題材にニューラルネットワークを学んでいくことにする。

In [0]:
# import packages
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# 仮想的なデータの作成
np.random.seed(0)
n = 40
X_train = np.random.randn(n, 2)

noise = 1.6 * np.random.randn(n)                        # 結果にノイズが含まれている
y_train = - X_train[:,0] + 2 * X_train[:,1] + noise > 0 # -x1 + 2 x2 > 0 をpositiveとして定義する

# 観測されたデータを散布図で示す
plt.scatter(X_train[:,0][y_train==True],X_train[:,1][y_train==True],  
            marker="o", label="positives")           # positive (y =  True) を o で表示
plt.scatter(X_train[:,0][y_train==False],X_train[:,1][y_train==False], 
            marker="x", label="negatives")           # negative (y = False) を x で表示
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')
plt.legend(loc="lower left")
plt.show()

## AI5.3 | PyTorchの利用

　この演習においては、ニューラルネットワークを構築するためのPythonライブラリとして、PyTorchを利用する（**補足資料 ※1**）。ニューラルネットワークのライブラリはバージョンが頻繁に更新されるため、再現性の担保のためにも常にバージョンを確認しておくと良い。


In [0]:
# Google Colabでは、最初からPyTorchがインストールされている
import torch

In [0]:
# ライブラリのバージョン確認
print(torch.__version__)
# -> 1.3.1（19.12.20 現在）

### AI5.3.1 | torch.tensorの利用

　torchの最も重要なものが`torch.tensor`である。
最も単純に説明すると、**NumPy配列`np.array`がさらに賢くなったものが`torch.tensor`**である。

In [0]:
A = torch.tensor([[1,2], [3,4]])
print(A)
print(A.dim())
print(A.shape) 
print(A[0,0]) # 1つの要素もtensorで表現される

　`torch.tensor`が最も効果を発揮するのは、勾配 (gradient) の計算である。以下のコードを実行して、$y = wx$ の $(x, w) = (1, 2)$ における勾配 $\frac{\partial y}{\partial x}, \frac{\partial y}{\partial w}$を計算してみる。

In [0]:
# requires_grad=Trueで勾配計算の対象を指定
# 入力値は必ず実数値でなければならない
# 整数値を入力するとエラーが発生する
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)

y = w * x
print(y)

# 勾配を計算
# ニューラルネットワークにおけるbackpropagation（誤差逆伝播）で利用するので
# 逆方向、という意味でbackwardというメソッド名になっている
y.backward()

# x, wに関する勾配を出力する
print("dy/dx (w=2) =", x.grad)
print("dy/dw (x=1) =", w.grad)

　これだけのコードで勾配の値を求めることができた。もう少し複雑な場合も実行してみる。

In [0]:
x = torch.tensor(1.0, requires_grad=True)

y = torch.exp(x*x)
y.backward()
print(x.grad)

　$y = e^{x^2}$ の時、 $\frac{d y}{d x} = 2xe^{x^2}$ なので、x=1の時の値は $2e = 5.43656...$ である。



　また、行列計算に対しても、勾配を求めることができる。
以下の例は、行列 $A$ と行列 $B$ の行列積のトレース $tr(AB)$ について、$A$ の勾配を求めている。（これは $\frac{\partial tr(AB)}{\partial A} = B^T$ となる）

In [0]:
mat_A = torch.tensor([[1.0, 2.0], [3.0, 4.0]], requires_grad=True)
mat_B = torch.tensor([[5.0, 6.0], [7.0, 8.0]])

mat_C = mat_A.mm(mat_B) # mm : matrixとmatrixの行列積を計算する
tr = mat_C.trace()
tr.backward() # 対角成分の和に関する勾配を計算
print(mat_A.grad) #mat_Aの勾配

　ちなみに、**`torch.tensor`に対しては、NumPyの関数は使わず、PyTorchの関数を利用しなければならない**。そうしないと、勾配が計算できなくなってしまうからだ（エラー表示としては、`torch.tensor`を`numpy.array`に直してからNumPy関数を使え、というメッセージになるので注意）。

In [0]:
# ダメな例：RuntimeErrorが発生する
import numpy as np
x = torch.tensor(1.0, requires_grad=True)

y = np.exp(x*x)
y.backward()
print(x.grad)

------
##### 課題 AI5.2

　シグモイド関数 $\sigma(a) = \frac{1}{1+e^{-a}}$ について、 $a=1$ の勾配 $\frac{d \sigma(a)}{d a}$ を`torch.tensor`を利用して求めよ。**ただし、`a.sigmoid()`は用いないこと**。

　課題提出時には、以下のコードの`__xxxxx__`部分を答えよ。



In [0]:
a = torch.tensor(1.0, requires_grad=True)
sigma = __xxxxx__
sigma.backward()
print(a.grad)

In [0]:
## 以下のコードと同じ結果が得られるべき
a = torch.tensor(1.0, requires_grad=True)
sigma = a.sigmoid()
sigma.backward()
print(a.grad)

-------

### AI5.3.2 | `torch.tensor`を利用した単層パーセプトロンの実装

　それでは、これまで使い方を簡単に見てきた`torch.tensor`を利用して、単層パーセプトロン (Single layer perceptron; SLP) の実装を行ってみる。


#### データセットの整形

　PyTorchでは、学習に用いる入力 $X$ および出力 $y$ について、以下の条件を満たす必要がある。

* 入力・出力はいずれも`torch.tensor`で表現されている
* 入力・出力はいずれも 行列である

特に、**出力の形式はscikit-learnの時と異なって行列**である必要があることに注意しよう。この条件を満たすように、5.2節で作成したデータ`train_X, train_y`をPyTorch用に変換する。



In [0]:
###### データの準備 ######

# yをベクトルから行列にする
Y_train = y_train[:, np.newaxis]

# NumPy配列をPyTorch用のデータ形式に変換する
# 学習のために、X, Yは実数値で表現する（dtype=torch.float）
# また、X, Yは学習対象ではないので requires_grad=Trueは行わない
X_torch = torch.tensor(X_train, dtype=torch.float)
Y_torch = torch.tensor(Y_train, dtype=torch.float)

　さらに、DatasetとDataLoaderというものを作る。これは、XとYの組を維持してくれるPyTorch特有の便利な仕組みである。

In [0]:
# XとYの組み合わせを保持してくれる便利なモノ
dataset = torch.utils.data.TensorDataset(X_torch, Y_torch)
# XとYの組み合わせを(batch_size)個ずつ出力する便利なモノ
# shuffle=True: 学習のたびにデータの順番を替えることで、データの並びに学習結果が影響されにくくする
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [0]:
## dataloaderはfor loopで配列のように使える
## shuffle=Trueの場合、実行するたびにデータの順番が変化する
for X, Y in dataloader:
  print("======")
  print("X is", X)
  print("Y is", Y)

#### 重み $w$ の定義

　そうしたら、次は単層パーセプトロンの重みを表現する $w$ を定義する。今回の場合は<font color="red">2</font>つの特徴量から<font color="blue">1</font>つの値を予測するので、 $w$ は <font color="red">2</font>×<font color="blue">1</font> の行列で表現されるべきだ（行列なので、コードでは大文字の`W`で表現する）。

In [0]:
# W の定義（乱数で初期化する）
torch.manual_seed(0) # 毎回同じ実行結果が得られるように乱数を固定する
W = torch.randn(2, 1, dtype=torch.float, requires_grad=True)
print(W)

#### 学習のコードの作成

　次に、学習のコードを記述する。

　コードを書くために、まず学習の流れを確認する（複雑なコードを書く場合は、このようにまず日本語で流れを整理すると、見通しが良くなる）。

1. 以下2.-6.をepoch数だけ繰り返す
2. 以下3.-6.をデータの個数だけ繰り返す
3. あるデータ $x$ について、現在の重み $w$ を使って、$y = 1$ の確率 $p(y=1|x, w)$ を計算する。
4. 実際の $y$ と予測結果を比較し、損失を計算する。
5. 損失を元に、誤差逆伝播法 (backpropagation) で勾配 $\frac{\partial L(w)}{\partial w}$を計算する。
6. 学習率 $\eta$ を使って、最急降下法と同一の更新式 $w^{(t+1)} = w^{(t)} - \eta \frac{\partial L(w)}{\partial w}$ で重み $w$ を更新する。


　○×の2値分類の場合には**シグモイド関数 (sigmoid function, $\sigma(a)$) を用いることで、予測値を0~1の確率値に変換**することができるので、この結果をもとに 3. $p(y=1|x,w)$ の計算を行う。

　また、4. の損失については、**分類予測の損失関数（誤差関数）は交差エントロピー誤差 (cross entropy loss, $H(y, \hat{y})$)** が一般的に使われる。

$\begin{eqnarray}
    \sigma(a) & = & \frac{1}{1+e^{-a}} \\
H(y, \hat{y}) & = & -y\log(\hat{y}) -(1-y)\log(1-\hat{y})
\end{eqnarray}$



　上記の学習の流れを **図 AI5.1** に、誤差逆伝播法の部分を **図 AI5.2** に示す。

![図 AI5.1](https://i.imgur.com/GfcPUMr.png)

**図 AI5.1 | 学習の大まかな流れ** `DataLoader(X, y, batch_size=1, shuffle=True)` 、データが3件しかない場合を例として示している。前述の箇条書き1-6を全て含めている。

![図 AI5.2](https://i.imgur.com/VsfdLcp.png)

**図 AI5.2 | 誤差逆伝播** 右から順番に（逆方向に）、局所の偏微分値を積算することで、勾配 $\frac{\partial l}{\partial w_1}, \frac{\partial l}{\partial w_2}$ を計算する。 $\Sigma, \sigma, H$ はそれぞれ総和、シグモイド関数、交差エントロピーを意味する。

---------

　以上の情報を全てコードにまとめると、以下のコードが出来上がる。

In [0]:
# 複数のデータXを入力としてそれぞれのデータが正例である確率 y_pred_proba を返す関数
# input: 行列 X, 行列 W
# output: 1次元配列 y_pred_proba
def predict_proba(X, W):
  y_pred_proba = torch.mm(X, W).sigmoid()
  return y_pred_proba

# 交差エントロピー誤差を計算
def cross_entropy(y_pred, y_true):
  ret = 0
  ret -=    y_true  * torch.log(    y_pred) # for y=1
  ret -= (1-y_true) * torch.log(1 - y_pred) # for y=0
  return ret

def train(X, Y, W_original, eta=0.01, epoch = 100):
  torch.manual_seed(0) ## 毎回同じ結果になるようにする
  # 入力として与えたWそのものを書き換えないようにするおまじない
  W = W_original.clone().detach().requires_grad_(True)

  # 1. epoch数だけ繰り返す
  for t in range(epoch):
    # 2. データの個数だけ繰り返す
    for data_X, data_Y in dataloader: # dataloaderからX, Yを同時に取り出す
      # 3. p(y=1|x,w)を求める
      y_proba = predict_proba(data_X, W)
      # 4. 損失の計算
      loss = cross_entropy(y_proba, data_Y)
      # 5. 誤差逆伝播法の実施（図AI5.2）
      # これによってW.gradが計算される
      loss.backward()

      # 勾配計算を行わないように一時的に（with構文の範囲だけ）制限する
      with torch.no_grad(): 
        # 6. 計算された勾配と学習率 eta を使って重みを更新
        W -= eta * W.grad 
        W.grad.zero_() # W.gradの中身を0に戻す
  return W

　ところで、このように**一部のデータの損失から重み $w$ を繰り返し更新する方法を確率的勾配降下法 (stochastic gradient descent; SGD)** と呼ぶ。

#### 学習の実行と学習結果の確認

　それでは、先ほど作成した `train()` 関数を用いて学習を実行してみるのだが、その前に、未学習の時点での予測正解率を確認してみよう。

　データの予測は、以下のコードで行うことができる。

In [0]:
count_ok  = 0
count_all = 0

for data_X, data_Y in dataloader:
  y_pred_proba = predict_proba(data_X, W) # W は学習前の重み
  y_pred = y_pred_proba > 0.5
  result = y_pred == data_Y
  count_ok += result.numpy()[0,0] # true -> 1, false -> 0 となる
print(count_ok, "/", len(Y_torch))

　柳澤がやった環境では、14/40件のみが予測に成功した。○か×かを予測するだけなので、ランダムにあてずっぽうを言うよりも精度が悪い状況だ。

　それでは、気を取り直して学習を行う。これまで書いてきた関数 `train()` を利用することで、簡単に学習が可能なはずである。

In [0]:
W_hat = train(X_torch, Y_torch, W)
print(W_hat)

　学習が行われ、`W_hat`が作成された。
これを元に、学習で利用したデータを予測してみる。

In [0]:
count_ok  = 0
count_all = 0

for data_X, data_Y in dataloader:
  y_pred_proba = predict_proba(data_X, W_hat)
  y_pred = y_pred_proba > 0.5
  result = y_pred == data_Y
  count_ok += result.numpy()[0,0] # true -> 1, false -> 0 となる
print(count_ok, "/", len(Y_torch))

　40個のデータのうち、31個は正しく予測ができるようになった。ちゃんと学習できているようだ。

------

##### 課題 AI5.3

　$x_{new} = (1,0)$ という新しい入力を与えた時の、`y_proba`の値を計算せよ。また、その結果から、この $x$ はどちらのクラスに属すると予想されるか答えよ。（ヒント：`torch.tensor`を準備する際に、`dtype=torch.float`を忘れないようにせよ）

------

### AI5.3.3 | PyTorchに準備されている「層」を利用した単層パーセプトロンの実装

　以上のコードは講義の内容の理解を深めるために行ってきた。しかし、実際には、PyTorchは様々なネットワーク構造や損失関数、重み $w$ の最適化手法を簡単に使うことができる。

　シグモイド関数と交差エントロピー誤差による単層パーセプトロンは以下のように実装できる（先ほどと違い、**図 AI5.2** の右図のように定数項  `bias` が含まれていることに注意してほしい）。

![図 AI5.2](https://i.imgur.com/De4HsRh.png)

**図 AI5.2 | 単層パーセプトロンにおける定数項（bias項）の有無**

In [0]:
import torch

In [0]:
###### 単層パーセプトロン (Single Layer Perceptron; SLP) の定義 ######
torch.manual_seed(0) # 同じ結果が出るようにする
slp = torch.nn.Sequential(
  torch.nn.Linear(2, 1, bias=True),   # Linearの中で重み w が定義され、それが更新される
)

# torch.nn.Linear(2, 1) だけでもbias=Trueになる

In [0]:
###### 重み w の更新に関する設定 ######

# 交差エントロピー誤差を計算する関数
# Binary Cross Entropy がBCEと略されている
loss_fn = torch.nn.BCEWithLogitsLoss()

# 確率的勾配降下法 (SGD) によるwの最適化
# lr（learning rate、学習率）がetaに対応する
optimizer = torch.optim.SGD(slp.parameters(), lr=0.1)

　AI5.3.2節では重み `W` を明示的に定義したが、PyTorchの実装では **`torch.nn.Linear` の中に `W` に対応する `weight` や `bias` が存在している**。初期状態ではどのような値になっているか、確認してみる。

In [0]:
print(slp.state_dict())

`0.weight`などと書かれているが、これは、0層目（一番最初の層）の重み $w$ という意味である。また、`0.bias`とは一番最初の層のバイアス項である。この層は $y = w^t \cdot x + b$ を計算していることになる。

　それでは、次に示すコードを用いて学習を行ってみる。 AI5.3.2節では `W -= eta * W.grad` によって `W` を更新したが、PyTorchのoptimizerを用いる場合には **`optimizer.step()` を実行することで `W` に対応する `weight` や `bias` が更新される**。

In [0]:
###### 学習の実行 #######

nEpoch = 50
for t in range(nEpoch):
  for data_X, data_Y in dataloader:
    Y_pred_proba = slp(data_X)           # 予測を実施
    loss = loss_fn(Y_pred_proba, data_Y) # 損失関数を計算
    #print(t, loss.item())

    optimizer.zero_grad() # 傾きの計算をするための初期化
    loss.backward()       # 各wについて、誤差逆伝播法で dL(w)/dw を計算
    optimizer.step()      # SGDの更新式に従いwを更新

  print(t, slp.state_dict())   # 学習の進行を確認するためにweightとbiasを出力してみる

　これで学習を行うことができた。この学習済みの `slp` を用いて、予測を行ってみよう。

In [0]:
###### 予測精度の計算 ######

# 複数の予測値と複数の正解値から予測精度を計算する
# True -> 1, False -> 0 と処理されることを利用すると
# mean()で予測正解率を計算できる
def accuracy(Y_pred, Y_true):
  result = (Y_pred == Y_true)
  return result.numpy().mean()

Y_pred_proba = slp(X_torch).sigmoid() # 予測の実施（sigmoid()によって確率を計算）
Y_pred = Y_pred_proba > 0.5           # 確率が0.5以上かそれ未満かで○×を予測
print(Y_pred_proba)
print(accuracy(Y_pred, Y_torch))      # 予測精度を計算

　これによりそれぞれのデータ `X` に対する正例である確率を計算し、その確率にしたがって〇/×の予測を行うことができた。

　ところで、ニューラルネットワークの学習では、1つのデータごとに更新するのではなく、**複数のデータごとに $w$ の勾配の平均を計算し、更新を行うミニバッチ (mini-batch)** という仕組みが通常用いられる。一度に学習するデータ数を示す `batch_size` を大きくすることで、**計算速度を向上させ**、**異常な値を持つデータの影響を比較的少なくする**ことができる。

　先ほど実行したコードで、DataLoaderの定義の部分に `batch_size=1` という記述があるが、これを `batch_size=4` とすると、4つのデータに基づいて $w$ を更新するようになる。


In [0]:
### batch_size = 4に変更してみる
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

In [0]:
## dataloaderの出力を再度確認してみる

## dataloaderはfor loopで配列のように使える
## shuffle=Trueの場合、実行するたびにデータの順番が変化する
for X, Y in dataloader:
  print("======")
  print("X is", X)
  print("Y is", Y)

　たしかに、dataloaderは4つのデータを同時に出力していることが確認できた。それでは、先ほどと同様に学習を行ってみよう。

In [0]:
###### 単層パーセプトロン (Single Layer Perceptron; SLP) の定義 ######
torch.manual_seed(0) # 何度実行しても同じ結果が出るようにする
slp = torch.nn.Sequential(
  torch.nn.Linear(2, 1, bias=True), 
)

###### 重みの更新に関する設定 ######
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(slp.parameters(), lr=0.1) # slpのwを更新することを指定する

In [0]:
###### 学習の実行 #######
nEpoch = 50
for t in range(nEpoch):
  for data_X, data_Y in dataloader:
    Y_pred_proba = slp(data_X)           # 予測を実施
    loss = loss_fn(Y_pred_proba, data_Y) # 損失関数を計算
    #print(t, loss.item())

    optimizer.zero_grad() # 傾きの計算をするための初期化
    loss.backward()       # 各wについて、誤差逆伝播法で dL(w)/dw を計算
    optimizer.step()      # SGDの更新式に従いwを更新

In [0]:
Y_pred_proba = slp(X_torch).sigmoid() # 予測の実施
Y_pred = Y_pred_proba > 0.5           # 確率が0.5以上かそれ未満かで○×を予測
print(accuracy(Y_pred, Y_torch))      # 予測精度を計算

　バッチサイズは $2^n$ を使うことが慣例的に多く、データ数が万単位で存在するのであれば128, 256, 512、場合によっては1024以上の値にする。

　最後に、この単層パーセプトロンのモデル `slp` の予測結果が、どのように $x_1, x_2$ に依存しているか見てみよう。以下の `draw()` は$x_1, x_2$の空間に予測された正例（〇）確率を等高線で表示する関数になっている。

In [0]:
import matplotlib.pyplot as plt
import numpy as np

def draw(aModel, X, y):
  # `aModel` を用いてグラデーションを描画する
  xx1 = np.arange(-3,3,0.1)
  xx2 = np.arange(-3,3,0.1)
  XX1,XX2 = np.meshgrid(xx1, xx2)
  XX = torch.tensor(
      np.hstack([XX1.reshape(-1,1),
                 XX2.reshape(-1,1)]),
      dtype=torch.float
  )
  YY = aModel(XX).sigmoid().detach().numpy().reshape(*XX1.shape)
  plt.contourf(XX1, XX2, YY, levels=[0.0,0.25,0.5,0.75,1.00])
  plt.colorbar()

  # 学習に用いたデータ点のプロット
  plt.scatter(X[:,0][y==True],  X[:,1][y==True],  marker="o", label="positives") 
  plt.scatter(X[:,0][y==False], X[:,1][y==False], marker="x", label="negatives") 

  plt.xlabel('$x_1$')
  plt.ylabel('$x_2$')
  plt.show()

In [0]:
draw(slp, X_train, y_train) # 単層パーセプトロンの学習結果を描画

　結果はどうなったであろうか。直線的な等高線が描画されたはずである。このように単層パーセプトロンは、基盤人工知能演習 第4回で学習したロジスティック回帰と同様に**線形モデル**である。

--------
##### 課題 AI5.4

　次回の基盤人工知能では、多層パーセプトロン (Multi layer perceptron) を学ぶのだが、先にモデルを構築してみよう。

　以下に示す多層パーセプトロンモデルについて、学習を行い、同様に等高線を描き、等高線が直線であるか曲線であるかを答えよ。

In [0]:
# 多層パーセプトロン (Multi Layer Perceptron; MLP)
# 間にSigmoid()を入れることを忘れずに
torch.manual_seed(0) # 結果を固定させる

mlp = torch.nn.Sequential(
  torch.nn.Linear(2, 2),   # 2変数→2変数
  torch.nn.Sigmoid(),
  torch.nn.Linear(2, 1),   # 2変数→1変数
)

loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr=0.1) # ここを mlp.paramters()に変更することを忘れずに

In [0]:
###### 学習の実行 ######
nEpoch = 250 # epoch数を増やした
for t in range(nEpoch):
  for data_X, data_Y in dataloader:

    # ここに学習のコードを実装せよ


In [0]:
###### 等高線の描画 ######
draw(mlp, X_train, y_train) # 多層パーセプトロンの学習結果を描画

##### 課題 AI5.5（発展）

　中間層に`Sigmoid()`を入れ忘れてしまった多層パーセプトロンの学習を行った時の等高線を確認せよ。直線的だろうか曲線だろうか。なぜこのようになるのか、考察せよ。

In [0]:
# sigmoidを入れ忘れた多層パーセプトロン
torch.manual_seed(0) # 結果を固定させる

mlp_wo_sigmoid = torch.nn.Sequential(
  torch.nn.Linear(2, 2),   # 2変数→2変数
  torch.nn.Linear(2, 1),   # 2変数→1変数
)

---------------------

## AI5.4 | GPU (Graphic Processing Unit) を用いた計算

　昨今のニューラルネットワークの隆盛に一役買っているのが、**GPU (Graphic Processing Unit) という部品**である。 この部品は Graphic の名前の通り、本来は**画面描画用のパーツ**である。

![GPUの例](https://i.imgur.com/J3toOw5.jpg)

このパーツを天体や流体、タンパク質などのシミュレーションなど、描画以外に転用することで、計算を高速化することができる。このGPUを使うことで、**ニューラルネットワークの計算の（数倍から10倍以上の）高速化**が達成できる。
Google Colaboratoryでは、GPUを無料で使うことができる（**※2**）ので、これを用いた計算を行ってみる。

　まず、**GPUがあるとしたらどこに存在するのか**を知る必要がある。これは `torch.device()` 関数で変数に入れることができる（実際にGPUがあるかどうかは確認していないので注意。今回のGoogle Colaboratoryでは、GPUが存在することを前提として構わない）。

In [0]:
import torch

###### GPUの存在を教える ######
# cudaはGPU上で様々な計算するためのプラットフォーム（ライブラリみたいなもの）の名前
device_GPU = torch.device("cuda:0") 

　GPUの場所を特定したら、**単層パーセプトロンのモデル**と**`X, y`のデータ**を**GPUへ転送**しながら学習を行う。

In [0]:
torch.manual_seed(0) # 結果を固定させる

###### モデル（単層パーセプトロン）の定義 ######
slp = torch.nn.Sequential(
  torch.nn.Linear(2, 1),   # w_1 x_1 + w_2 x_2 + b に対応、Linearの中の w が更新される
)
slp.to(device_GPU)  # GPUにモデルを転送

###### 重みの更新に関する設定 ######
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(slp.parameters(), lr=0.1) # slpのwを更新することを指定する

In [0]:
###### 学習の実行 #######

# batch_size = 4
dataloader = torch.utils.data.DataLoader(dataset, batch_size=4, shuffle=True)

# 学習は高速になる…はずなのだが？？
nEpoch = 50
for t in range(nEpoch):
  for data_X, data_Y in dataloader:
    data_X = data_X.to(device_GPU)        # GPUにデータを転送
    data_Y = data_Y.to(device_GPU)        # GPUにデータを転送
    Y_pred_proba = slp(data_X)
    loss = loss_fn(Y_pred_proba, data_Y)  
    #print(t, loss.item())

    optimizer.zero_grad() # 傾きの計算をするための初期化
    loss.backward()       # 各wについて、誤差逆伝播法で dL(w)/dw を計算
    optimizer.step()      # SGDの更新式に従いwを更新

　これで、GPUを使った学習が行われた。次に予測を行うが、予測を行う場合は次の手順を踏む。

- `X` をGPUに転送する
- GPU上で予測を行う
- 予測結果をGPUから計算機に転送する

最後の「GPUから計算機に転送」は `cpu()` を用いて実行することができる。

In [0]:
X_torch = X_torch.to(device_GPU)      # 予測時はXだけGPUに転送
Y_pred_proba = slp(X_torch).sigmoid() # 予測の実施
Y_pred_proba = Y_pred_proba.cpu()     # 予測結果をGPUから計算機に転送
Y_pred = Y_pred_proba > 0.5           # 確率が0.5以上かそれ未満かで○×を予測
print(accuracy(Y_pred, Y_torch))      # 予測精度を計算

　これで、GPUを使った学習と予測を実施できるようになった。**`to(device_GPU)`で計算機→GPUの転送、`cpu()`でGPU→計算機の転送**と覚えておこう。

　…ところで、学習速度はどのように変わっただろうか。GPUを使ってどの程度高速化されただろうか。実は `batch_site=4` である今回のケースだと、バッチサイズが小さく、GPUを効果的に使うことができない。
次回以降はMNISTと呼ばれる手書き文字画像の分類を行うが、その時にはGPUを使わないと話にならないくらい遅くなってしまうので、先もって教えておいた、と理解してほしい。


# レポート提出について



## レポートの提出方法
　レポートは**答案テンプレートを用い**、**1つのファイル (.doc, .docx, .pdf)** にまとめ、**学籍番号と氏名を確認の上**、**1/16 15:00 (次回 基盤人工知能演習) までに東工大ポータルのOCW-iから提出**すること。
ファイルのアップロード後、OCW-iで「提出済」というアイコンが表示されていることを必ず確認すること。それ以外の場合は未提出扱いとなるので十分注意すること。
また、締め切りを過ぎるとファイルの提出ができないため、時間に余裕を持って提出を行うこと。

## 答案テンプレート

```
学籍番号:
名前:

課題 AI5.1
__xxxxx__ = 
__yyyyy__ = 

課題 AI5.2
__xxxxx__ = 

課題 AI5.3
y_proba = 
x_new は { 正例 | 負例 } に属すると予想される。

課題 AI5.4
多層パーセプトロンの場合、等高線は { 直線 | 曲線 } を描く。

課題 AI5.5（発展）
等高線は { 直線 | 曲線 } を描く。
（考察を記述せよ）

```

# 補足資料

## ※1 | PyTorchのもうちょっと真面目な勉強資料

　今回の講義では、時間の都合でいろんな知識を飛ばして教えている。
より詳しく知りたい場合は、[PyTorchのtutorial](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) や [yunjeyによる非公式なtutorial](https://github.com/yunjey/pytorch-tutorial) を読み進めることをお勧めする。

　日本語の書籍も存在する（例：「現場で使える！PyTorch開発入門―深層学習モデルの作成とアプリケーションへの実装」翔泳社）が、ニューラルネットワーク（深層学習）はライブラリの仕様などが頻繁に変更されているため、書籍が古いと最新版では存在しない関数を利用している可能性もある。PythonやPyTorchのバージョンには十分注意して、実際にGoogle Colaboratory等で動かしながら学習すると良い。

## ※2 | Google ColaboratoryにおけるGPUの利用

　今回授業資料として配布したJupyter Notebookは**GPU利用設定がすでにONになっており**、追加の設定なくGPUを使うことができる。一方、Google Colaboratoryの初期設定はGPU利用がOFFになっているので、ONにする方法を紹介する（以下の資料は[Chainer tutorialの「1.3.5. GPUを利用する」](https://tutorials.chainer.org/ja/01_Welcome_to_Chainer_Tutorial.html#GPU-%E3%82%92%E4%BD%BF%E7%94%A8%E3%81%99%E3%82%8B)から引用・一部修正している）。

　まず、画面上部のタブの中の 「ランタイム」 (Runtime) をクリックし、「ランタイムのタイプを変更」 (Change runtime type) を選択する。

　すると、「ノートブックの設定」 (Notebook Settings) というものが表示されるはずである。この中の「ハードウェア アクセラレータ」 (Hardware Accelerator)をNoneからGPUに変更して、保存すればよい。

![GPUに変更](https://tutorials.chainer.org/ja/_images/01_08.png)